Roster id's

Model Training

In [15]:
import pandas as pd

# Load the primary train data
train_path = './MergedTrainData.csv'  # Replace with the actual file path
data = pd.read_csv(train_path)

# Assuming df is your DataFrame
# data_types = data.dtypes

# print(data_types)
print(data.shape)

(2637, 23)


In [16]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode categorical variables
categorical_cols = ['team1', 'team2', 'toss winner', 'toss decision', 'venue', 'city', 'lighting', 'series_name', 'season']
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

# drop column
data.drop(columns=['match_dt','winner'], inplace=True)
data.drop(columns=['team1_roster_ids', 'team2_roster_ids'], inplace=True)

# Display the updated data types
# print(data.dtypes)


In [17]:
# Check for missing values
missing_values = data.isnull().sum()
# print("Missing Values:\n", missing_values)

# Handle missing values if any
# For simplicity, let's fill missing values with the mean of each column
data.fillna(0, inplace=True)


In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV

# Define features (X) and target variable (y)
X = data.drop(columns=['winner_id'])
y = data['winner_id']

# Split the dataset into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and validation sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)


Training set shape: (2109, 18) (2109,)
Validation set shape: (528, 18) (528,)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 4: Model Selection and Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 50, 100],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from grid search
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Step 5: Model Evaluation with the best parameters
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

Fitting 3 folds for each of 972 candidates, totalling 2916 fits


c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
972 fits failed out of a total of 2916.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
188 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params

Best parameters found:  {'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Validation Accuracy: 0.7026515151515151


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Step 5: Model Evaluation with the best parameters
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_val)

# Accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Precision
precision = precision_score(y_val, y_pred, average='weighted')  # Use 'weighted' to account for class imbalance
print("Validation Precision:", precision)

# Recall
recall = recall_score(y_val, y_pred, average='weighted')
print("Validation Recall:", recall)

# F1-Score
f1 = f1_score(y_val, y_pred, average='weighted')
print("Validation F1-Score:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_val, y_pred)
print("Classification Report:\n", class_report)


Validation Accuracy: 0.7026515151515151
Validation Precision: 0.7352211761302672
Validation Recall: 0.7026515151515151
Validation F1-Score: 0.6932829564632985
Confusion Matrix:
 [[8 0 2 ... 0 0 0]
 [0 5 0 ... 0 0 0]
 [0 0 5 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 2]]
Classification Report:
               precision    recall  f1-score   support

          20       0.53      0.73      0.62        11
          27       0.71      0.83      0.77         6
          34       0.62      1.00      0.77         5
          41       1.00      0.88      0.93         8
          48       0.75      0.38      0.50         8
          55       0.67      0.75      0.71         8
          62       0.86      0.60      0.71        10
          69       1.00      1.00      1.00         5
          76       0.60      0.86      0.71         7
          90       0.00      0.00      0.00         1
         118       1.00      0.33      0.50         3
         146       0.67      

c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\rahul.k.a.jha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

Testing

In [20]:
import pandas as pd

# Load the primary train data
test_path = './mergedtest.csv'  # Replace with the actual file path
test_data = pd.read_csv(test_path)

In [21]:
print(test_data.head(0))

Empty DataFrame
Columns: [match id, team1, team1_id, team2, team2_id, toss winner, toss decision, venue, city, lighting, series_name, season, ground_id, team_count_50runs_last15, team_winp_last5, team1only_avg_runs_last15, team1_winp_team2_last15, ground_avg_runs_last15, dataset_type]
Index: []


In [22]:
# Separate the columns used for prediction
predict_columns = [
    'match id', 'team1', 'team1_id', 'team2', 'team2_id', 'toss winner', 'toss decision', 'venue', 'city', 
    'lighting', 'series_name', 'season', 'ground_id', 
    'team_count_50runs_last15', 'team_winp_last5', 'team1only_avg_runs_last15', 
    'team1_winp_team2_last15', 'ground_avg_runs_last15'
]

# Ensure 'dataset_type' is not in predict_columns
predict_columns = [col for col in predict_columns if col != 'dataset_type']

# Prepare the data for prediction (excluding 'dataset_type')
X_test = test_data[predict_columns]


In [23]:
# # Ensure the test dataset has the same feature columns as the training data
# X_test = test_data[X_train.columns]

# Make predictions on the test set
predictions = best_rf.predict(X_test)
prediction_scores = best_rf.predict_proba(X_test).max(axis=1)  # Get the max probability for the predicted class

# Output the predictions
# Prepare DataFrame
dep_var = pd.DataFrame({
    'match id': test_data['match id'],
    'dataset_type': test_data['dataset_type'],
    'winner_id':  predictions,
})

dep_var.to_csv('./dep_var.csv', index=False)

Primary Submission File

In [24]:
import numpy as np

# Get top 10 features and their values for each prediction
top_10_features = np.argsort(best_rf.feature_importances_)[-10:][::-1]

# Assuming `test_data` is your test DataFrame
test_data['win_pred_team_id'] = predictions
test_data['win_pred_score'] = prediction_scores


In [27]:
# Model and hyperparameters details
train_algorithm = 'RandomForest'
is_ensemble = 'no'
train_hps_trees = 300  # Example value
train_hps_depth = 20  # Example value
train_hps_lr = 0  # Example value for non-ensemble model

# Prepare DataFrame
primary_submission = pd.DataFrame({
    'match id': test_data['match id'],
    'dataset_type': test_data['dataset_type'],
    'win_pred_team_id': test_data['win_pred_team_id'],
    'win_pred_score': test_data['win_pred_score'],
    'train_algorithm': train_algorithm,
    'is_ensemble': is_ensemble,
    'train_hps_trees': train_hps_trees,
    'train_hps_depth': train_hps_depth,
    'train_hps_lr': train_hps_lr,
})

# Add top 10 feature values to the DataFrame
for i, feat_idx in enumerate(top_10_features):
    primary_submission[f'indep_feat_id{i+1}'] = X_test.iloc[:, feat_idx].values

# Save to CSV
primary_submission.to_csv('primary_submission.csv', index=False)

print("Primary submission file created successfully.")


Primary submission file created successfully.


Secondary Submission File

In [28]:
# Get feature importances
feature_importances = best_rf.feature_importances_

# Create a DataFrame for feature importances
features_df = pd.DataFrame({
    'feat_id': range(1, len(X.columns) + 1),
    'feat_name': X.columns,
    'model_feat_imp_train': feature_importances
})

# Rank the features by importance
features_df['feat_rank_train'] = features_df['model_feat_imp_train'].rank(ascending=False).astype(int)

# Add a placeholder for feature descriptions (you should replace this with actual descriptions)
features_df['feat_description'] = "Description of " + features_df['feat_name']

# Sort by feature importance rank
features_df = features_df.sort_values(by='feat_rank_train')

# Save to CSV
features_df.to_csv('secondary_submission.csv', index=False)

print("Secondary submission file created successfully.")


Secondary submission file created successfully.
